In [1]:
import duckdb
import pandas as pd

In [2]:
def get_qmt_quotes(
        ticker_symbol:str, 
        period:str='1m', 
        start_time:str=None, 
        end_time:str=None
    ) -> pd.DataFrame:
    db = duckdb.connect()
    if period not in ['1m', '5m', '1d']:
        raise ValueError(f"period must be one of ['1m', '5m', '1d'], but got {period}")
    
    if start_time is None:
        start_time = '20000101 00:00:00'

    if end_time is None:
        end_time = '20991231 00:00:00'
    if start_time > end_time:
        raise ValueError(f"start_time must be less than end_time, but got {start_time} and {end_time}")

    start_time = pd.to_datetime(start_time)
    end_time = pd.to_datetime(end_time)
    query = f"""
        SELECT * 
        FROM 
        read_parquet('D:/qmt_datadir/{period}/{ticker_symbol}/*')
        where 
        1=1
        and trade_time between '{start_time}' and '{end_time}'
    """
    return db.execute(query).fetchdf()


In [3]:
df = get_qmt_quotes('000001.SZ', '1m', start_time='20150101')

In [16]:
period = "1m"
ticker_symbol = "000001.SH"
target_period = '60'
target_bucket = f"{target_period} minutes"

con = duckdb.connect()
query = f"""
WITH market_data AS (
    SELECT
        *
    FROM
        read_parquet('D:/qmt_datadir/{period}/{ticker_symbol}/*.parquet')

),
grouped_data AS (
        SELECT
            ticker_symbol,
            time_bucket('{target_bucket}', trade_time) AS bucket_time,
            open,
            high,
            low,
            close,
            volume,
            amount,
            settelement_price,
            open_interest,
            preclose,
            suspend_flag
        FROM
            market_data
        ORDER BY
            trade_time
    ),
    aggregated_data AS (
        SELECT
            ticker_symbol,
            bucket_time,
            FIRST(open) AS open,
            MAX(high) AS high,
            MIN(low) AS low,
            LAST(close) AS close,
            SUM(volume) AS volume,
            SUM(amount) AS amount,
            LAST(settelement_price) AS settelement_price,
            LAST(open_interest) AS open_interest,
            LAST(preclose) AS preclose,
            LAST(suspend_flag) AS suspend_flag
        FROM
            grouped_data
        GROUP BY
            ticker_symbol,
            bucket_time
    )
    SELECT
        bucket_time AS trade_time,
        ticker_symbol,
        open,
        high,
        low,
        close,
        volume,
        amount,
        settelement_price,
        open_interest,
        preclose,
        suspend_flag
    FROM
        aggregated_data
    ORDER BY
        ticker_symbol,
        trade_time;
"""
df = con.execute(query).df()


In [3]:
import duckdb
import pandas as pd
con = duckdb.connect()
query = """
select *, strftime(time, '%Y%m') as year_month from read_parquet('d:/adjust_factor.parquet')
"""
df = con.execute(query).df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [8]:
for year_month, df_group in df.groupby('year_month'):
    print(year_month)
    temp = df_group.copy()
    temp.drop(['year_month'], axis=1, inplace=True)
    temp.to_parquet(f'D:/qmt_datadir/adjust_factor/{year_month}.parquet', compression='snappy')

200501
200502
200503
200504
200505
200506
200507
200508
200509
200510
200511
200512
200601
200602
200603
200604
200605
200606
200607
200608
200609
200610
200611
200612
200701
200702
200703
200704
200705
200706
200707
200708
200709
200710
200711
200712
200801
200802
200803
200804
200805
200806
200807
200808
200809
200810
200811
200812
200901
200902
200903
200904
200905
200906
200907
200908
200909
200910
200911
200912
201001
201002
201003
201004
201005
201006
201007
201008
201009
201010
201011
201012
201101
201102
201103
201104
201105
201106
201107
201108
201109
201110
201111
201112
201201
201202
201203
201204
201205
201206
201207
201208
201209
201210
201211
201212
201301
201302
201303
201304
201305
201306
201307
201308
201309
201310
201311
201312
201401
201402
201403
201404
201405
201406
201407
201408
201409
201410
201411
201412
201501
201502
201503
201504
201505
201506
201507
201508
201509
201510
201511
201512
201601
201602
201603
201604
201605
201606
201607
201608
201609
201610
201611